In [729]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Pré-processamento #

### Base e encoder manual ###

In [730]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [731]:
warnings.filterwarnings("ignore")

In [732]:
df_encod_manual = pd.DataFrame.copy(df)

In [733]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [734]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [735]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [736]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [737]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [738]:
previsores_label = df.iloc[:, 0:11].values

In [739]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [740]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [741]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [742]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [743]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [744]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [745]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# Previsores #
<span style="font-size: 13px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# **Regressão Logística** #

<span style="font-size: 14px;">
A regressão logística é uma técnica de modelagem estatística amplamente utilizada em machine learning para problemas de classificação binária.</br> Ela é aplicada quando queremos prever a probabilidade de uma observação pertencer a uma determinada categoria.
</br>

**Conceitos Básicos**

A regressão logística utiliza a função logística (também conhecida como função sigmóide) para modelar a relação entre as variáveis independentes e a variável dependente.</br> A função logística é dada por:
</span>

<span style="font-size: 18px;">

$ P(Y=1|X) $ = $ \frac{1}{1 + e^{-(\beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_pX_p)}} $

</span>

<span style="font-size: 15px;">
onde: </br>

- $ ( P(Y=1|X) ) $ é a probabilidade condicional de $ ( Y ) $ ser igual a 1 dado $ ( X ) $.
- $ ( e ) $ é a base do logaritmo natural.
- $ ( \beta_0, \beta_1, \beta_2, ..., \beta_p ) $ são os coeficientes a serem estimados.
- $ ( X_1, X_2, ..., X_p ) $ são as variáveis independentes. </br></br>

</span>


<span style="font-size: 14px;">

**Parâmetros**

- **C**: Parâmetro de regularização que controla a força da regularização. Quanto menor o valor de C, mais forte é a regularização. </br>
- **Solver**: Algoritmo utilizado para otimizar os coeficientes do modelo. Alguns dos solvers comuns são 'liblinear', 'newton-cg', 'lbfgs', 'sag' e 'saga'. </br>
- **Penalty**: Tipo de regularização a ser aplicada. Pode ser 'l1' (regularização L1), 'l2' (regularização L2) ou 'none' (sem regularização). </br>
- **Max_iter**: Número máximo de iterações para os solvers convergirem.


A regressão logística é uma técnica poderosa e versátil que é amplamente utilizada em problemas de classificação binária devido à sua interpretabilidade e eficiência computacional.
</span>

## Criando e treinando o modelo ##

In [746]:
logistic = LogisticRegression(
    random_state = 5, max_iter = 10,
    penalty = 'elasticnet', tol = 0.0001, 
    C = 1, solver = 'saga', l1_ratio = 0.5
)

logistic.fit(x_train, y_train);

In [747]:
logistic.intercept_

array([0.27475891])

In [748]:
logistic.coef_

array([[ 3.09540946e-01, -3.09540946e-01,  5.67115238e-01,
        -3.64989176e-01, -1.27571591e-01, -5.68085468e-02,
         5.56406960e-02,  0.00000000e+00, -4.65629198e-02,
        -2.17242355e-01,  2.17242355e-01, -7.23678072e-04,
         6.33609452e-01, -5.79711048e-01,  1.76176492e-01,
         1.70674543e-04,  1.82412790e-01,  4.27163490e-01,
        -2.20671450e-01,  2.91480009e-01]])

In [749]:
previsoes_test = logistic.predict(x_test)

In [750]:
previsoes_train = logistic.predict(x_train)

## Métricas ##

### Dados de Teste ###

In [751]:
acuracia_test = accuracy_score(y_test, previsoes_test) * 100.0
print("Acurácia teste: %.2f%%" % acuracia_test)

Acurácia teste: 86.23%


In [752]:
print("Matriz de confusão teste: \n", confusion_matrix(y_test, previsoes_test))

Matriz de confusão teste: 
 [[102  19]
 [ 19 136]]


In [753]:
print("Relatório de teste: \n", classification_report(y_test, previsoes_test))

Relatório de teste: 
               precision    recall  f1-score   support

           0       0.84      0.84      0.84       121
           1       0.88      0.88      0.88       155

    accuracy                           0.86       276
   macro avg       0.86      0.86      0.86       276
weighted avg       0.86      0.86      0.86       276



### Dados de Treino ###

In [754]:
acuracia_train = accuracy_score(y_train, previsoes_train) * 100.0
print("Acurácia treino: %.2f%%" % acuracia_train)

Acurácia treino: 85.80%


In [755]:
print("Matriz de confusão treino: \n", confusion_matrix(y_train, previsoes_train))

Matriz de confusão treino: 
 [[237  52]
 [ 39 313]]


In [756]:
print("Relatório de treino: \n", classification_report(y_train, previsoes_train))

Relatório de treino: 
               precision    recall  f1-score   support

           0       0.86      0.82      0.84       289
           1       0.86      0.89      0.87       352

    accuracy                           0.86       641
   macro avg       0.86      0.85      0.86       641
weighted avg       0.86      0.86      0.86       641

